# Feature extraction

### Imports Modules

In [3]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import os
import PIL.Image as Image
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"]= "-1"


C:\Users\lyeso\anaconda3\envs\exts-ml\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Create Image Generator

In [4]:
# Create image generator
train_generator = ImageDataGenerator(
    rescale=1/255)
test_generator = ImageDataGenerator(rescale=1/255)

In [10]:
# Train, validation and test sets

trainset = train_generator.flow_from_directory(
    os.path.join('Course Project - SwissRoads', 'train'), batch_size =500, target_size=(299, 299),
    shuffle=False, color_mode='rgb', class_mode = 'sparse')
validset = train_generator.flow_from_directory(
    os.path.join('Course Project - SwissRoads', 'valid'),batch_size =500, target_size=(299, 299),
    shuffle=False, color_mode='rgb', class_mode = 'sparse')
testset = test_generator.flow_from_directory(
    os.path.join('Course Project - SwissRoads', 'test'),batch_size =500, target_size=(299, 299),
    shuffle=False,color_mode='rgb', class_mode = 'sparse')

Found 280 images belonging to 6 classes.
Found 139 images belonging to 6 classes.
Found 50 images belonging to 6 classes.


In [11]:
train_imgs, train_labels = trainset.next()
valid_imgs, valid_labels = validset.next()
test_imgs, test_labels = testset.next()

print('Train images:', train_imgs.shape) 
print('Train labels:', train_labels.shape) 
print('Valid images:', valid_imgs.shape) 
print('Valid labels:', valid_labels.shape) 
print('Test images:', test_imgs.shape) 
print('Test labels:', test_labels.shape) 

Train images: (280, 299, 299, 3)
Train labels: (280, 6)
Valid images: (139, 299, 299, 3)
Valid labels: (139, 6)
Test images: (50, 299, 299, 3)
Test labels: (50, 6)


In [12]:
trainset.class_indices

{'bike': 0, 'car': 1, 'motorcycle': 2, 'other': 3, 'truck': 4, 'van': 5}

In [14]:
# Save the filenames

train_files = trainset.filenames
valid_files = validset.filenames
test_files = testset.filenames

### Create graphs for features extraction

In [15]:
# Create graph
img_graph = tf.Graph()

with img_graph.as_default():
    # Download module
    module_url = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/3"
    feature_extractor = hub.Module(module_url)

    # Create input placeholder
    input_imgs = tf.placeholder(dtype=tf.float32, shape=[None, 299, 299, 3])

    # A node with the features
    imgs_features = feature_extractor(input_imgs)

    # Collect initializers
    init_op = tf.group([
        tf.global_variables_initializer(), tf.tables_initializer()
    ])

img_graph.finalize() # Good practice: make the graph "read-only"

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


### Extract features

In [16]:
# Create a session
sess = tf.Session(graph=img_graph)

# Initialize it
sess.run(init_op)

# Extract features
features_train = sess.run(imgs_features, feed_dict={input_imgs: train_imgs})
print(features_train.shape)

features_valid = sess.run(imgs_features, feed_dict={input_imgs: valid_imgs})
print(features_valid.shape)

features_test = sess.run(imgs_features, feed_dict={input_imgs: test_imgs})
print(features_test.shape)



(280, 2048)
(139, 2048)
(50, 2048)


### Save 3 NPZ files

In [18]:
np.savez('train.npz', features = features_train , labels = train_labels, pixels = train_imgs, names = train_files)
np.savez('valid.npz', features = features_valid , labels = valid_labels, pixels = valid_imgs , names = valid_files)
np.savez('test.npz', features = features_test , labels = test_labels, pixels = test_imgs, names = test_files)